# Auto inssurance 

In [ ]:
%matplotlib inline 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from collections import Counter
from scipy import interpolate

In [ ]:
df = pd.read_excel('https://query.data.world/s/yg4pp7vm5dhoj6pj4xjm43gcg73ujq',header=3)
df.drop(columns=['Timestamp','PostalCode', 'Counter', 'CompanyEmployeesOverall', 'PopulationOfLargestCityWithin20Miles'], inplace=True)
df['SalaryUSD'] = list(map(float, [str(ch).replace(" ", "").replace(",", ".") for ch in list(df.SalaryUSD)]))
df.columns

Index(['Survey Year', 'SalaryUSD', 'Country', 'PrimaryDatabase',
       'YearsWithThisDatabase', 'OtherDatabases', 'EmploymentStatus',
       'JobTitle', 'ManageStaff', 'YearsWithThisTypeOfJob', 'HowManyCompanies',
       'OtherPeopleOnYourTeam', 'DatabaseServers', 'Education',
       'EducationIsComputerRelated', 'Certifications', 'HoursWorkedPerWeek',
       'TelecommuteDaysPerWeek', 'EmploymentSector', 'LookingForAnotherJob',
       'CareerPlansThisYear', 'Gender', 'OtherJobDuties',
       'KindsOfTasksPerformed'],
      dtype='object')

In [ ]:
df.head(3)

,Survey Year,SalaryUSD,Country,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,DatabaseServers,Education,EducationIsComputerRelated,Certifications,HoursWorkedPerWeek,TelecommuteDaysPerWeek,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed
0,2017,200000.0,United States,Microsoft SQL Server,10,MySQL/MariaDB,Full time employee,DBA,No,5,Not Asked,2,350,Masters,No,"Yes, and they're currently valid",45,1,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked
1,2017,61515.0,United Kingdom,Microsoft SQL Server,15,"Oracle, PostgreSQL",Full time employee,DBA,No,3,Not Asked,1,40,None (no degree completed),NaN,"No, I never have",35,2,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked
2,2017,95000.0,Germany,Microsoft SQL Server,5,"Oracle, MySQL/MariaDB, Informix",Full time employee,Other,Yes,25,Not Asked,2,100,Masters,Yes,"Yes, but they expired",45,"None, or less than 1 day per week",Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked


In [ ]:
df.loc[:,'OtherJobDuties']=df.loc[:,'OtherJobDuties'].fillna('Unkown')

In [ ]:
def get_elements(key): 
  L = key.split(",")
  Result = []
  i = 0 
  while i < len(L):
    word = L[i]
    if word.find("(") != -1: 
      while word.find(")") == -1 : 
        i += 1 
        word = word +","+ L[i]
    i += 1
    Result.append(word) 
  return Result

def get_elts_count(X, col): 
  elts_counter = Counter(X[col])
  elts_pairs = {}
  for key, value in elts_counter.items():
    if type(key) != str:
      continue
    elts = get_elements(key)
    for elt in elts:
      if elt not in elts_pairs.keys(): 
        elts_pairs[elt] = value
      else:
        elts_pairs[elt] += value 
  return elts_pairs

In [ ]:
job_duties_counter = Counter(df.OtherJobDuties)
job_duties_pairs = {}

for key, value in job_duties_counter.items():
  duties = get_elements(key)
  for dutie in duties:
    if dutie not in job_duties_pairs.keys(): 
      job_duties_pairs[dutie] = [value]
    else:
      job_duties_pairs[dutie][0] += value 

In [ ]:
df.loc[:,'KindsOfTasksPerformed'] = df.loc[:,'KindsOfTasksPerformed'].fillna('Unkown')
L = get_elts_count(df, "KindsOfTasksPerformed")
L = [(key,value) for key, value  in L.items()]
L =  sorted(L, key=lambda tup: tup[1], reverse= True)

In [ ]:
def get_dummy_jobs(X_df, num): # Here X_df is composed of a single column
  feature_array = np.zeros((len(X_df.index), num))
  col_name = X_df.columns[0]
  elts_pairs  = get_elts_count(X_df, col_name)
  elts_pairs = [(key,value) for key, value  in elts_pairs.items()]
  elts_pairs =  sorted(elts_pairs, key=lambda tup: tup[1], reverse= True)
  elts_kept = [pair[0]  for pair in elts_pairs[:num] ]
  X_df = X_df[col_name]
  indexes = list(X_df.index.values)
 
  for p in range(len(X_df)):
    i = indexes[p]
    if (type(X_df[i]) != str):
      for  j in range(len(elts_kept)):
        feature_array[p][j] = np.nan
      continue
    elts = get_elements(X_df[i])
    for  j in range(len(elts_kept)): 
      if elts_kept[j] in elts : 
        feature_array[p][j] = 1 
  return feature_array

In [ ]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

transformer_dummy_jobs = FunctionTransformer(
    lambda X_df: get_dummy_jobs(X_df, 1)
)


transformer = make_column_transformer(
    (transformer_dummy_jobs, ['OtherJobDuties']),
    (transformer_dummy_jobs, ['KindsOfTasksPerformed']),
)
df.loc[:,'KindsOfTasksPerformed']=df.loc[:,'KindsOfTasksPerformed'].fillna('Unkown')
df.loc[:,'OtherJobDuties']=df.loc[:,'OtherJobDuties'].fillna('Unkown')
X_array = transformer.fit_transform(df)
print(np.sum(X_array))
X_array

get_dummy_jobs start
done
get_dummy_jobs start
done
5927.0


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 0.],
       [0., 1.],
       [0., 0.]])

In [ ]:
def get_top_jobs(X_df, num): # Here X_df is composed of a single column
  feature_array = np.zeros((len(X_df), num))
  col_name = X_df.columns[0]
  elts_pairs = Counter(X_df[col_name])
  elts_pairs = [(key,value) for key, value  in elts_pairs.items()]
  elts_pairs =  sorted(elts_pairs, key=lambda tup: tup[1], reverse= True)
  elts_kept = [pair[0]  for pair in elts_pairs[:num - 1] ]
  if 'Other' in elts_kept: 
    elts_kept.append(elts_pairs[num][0])
  else : 
    elts_kept.append('Other')
  Other_index = elts_kept.index('Other')
  X_df = X_df[col_name]
  indexes = list(X_df.index.values)
 
  for p in range(len(X_df)):
    i = indexes[p]
    if (type(X_df[i]) != str):
      for  j in range(len(elts_kept)):
        feature_array[p][j] = np.nan
      continue
    elt = X_df[i]
    if  elt in elts_kept:
      for j in range(num): 
        if elt == elts_kept[j]: 
          feature_array[p][j] = 1
          break
    else: 
      feature_array[p][num-1] = 1
  return feature_array

In [ ]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

transformer_JobTitle = FunctionTransformer(
    lambda X_df: get_top_jobs(X_df, 1)
)

transformer = make_column_transformer(
    (transformer_JobTitle, ['JobTitle']),
)
df.loc[:,'JobTitle']=df.loc[:,'JobTitle'].fillna('Other')
X_array = transformer.fit_transform(df)
print(np.sum(X_array))
X_array

get_job_dummy start
done
6893.0


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [ ]:
C = Counter(df.CareerPlansThisYear)
[key for key in C.keys()]

['Not Asked',
 'Stay with the same employer, same role',
 'Prefer not to say',
 'Stay with the same employer,  but change roles',
 'Change both employers and roles',
 'Stay with the same role, but change employers']

In [ ]:
def encode_career(X_df): 
  col_name = X_df.columns[0]
  counter = Counter(X_df[col_name])
  counter = [key for key in counter.keys()]
  feature_array = np.zeros((len(X_df), len(counter)))
  X_df = X_df[col_name]
  indexes = list(X_df.index.values)

  for p in range(len(X_df)):
    i = indexes[p]
    if (type(X_df[i]) != str):
      for  j in range(len(counter)):
        feature_array[p][j] = np.nan
      continue
    j = counter.index(X_df[i])
    feature_array[p][j] = 1
  return feature_array

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer


job_transformer = Pipeline(steps=[
    ('transform', transformer_dummy_jobs),
    ('imputer', SimpleImputer(strategy= 'most_frequent'))])


jobTitle_transfomer = Pipeline(steps=[
    ('transform', transformer_JobTitle),
    ('imputer', SimpleImputer(strategy= 'most_frequent'))])

check = FunctionTransformer(
    lambda X: print("Start One hot encoder")
)

carreer_transformer = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder()),
    ('imputer', SimpleImputer(strategy= 'most_frequent')),          
])

preprocessor = ColumnTransformer(
    transformers=[
        ('0', job_transformer, ['OtherJobDuties']),
        ('1', job_transformer, ['KindsOfTasksPerformed']),
        ('2', jobTitle_transfomer, ['JobTitle']),
        ('3', carreer_transformer, ['CareerPlansThisYear']),
    ])

pipe = make_pipeline(
    preprocessor,
    RandomForestRegressor(max_depth=5, n_estimators=10000, criterion= 'mse', n_jobs= -1)
)
df = pd.read_excel('https://query.data.world/s/yg4pp7vm5dhoj6pj4xjm43gcg73ujq',header=3)
df.drop(columns=['Timestamp','PostalCode', 'Counter', 'CompanyEmployeesOverall', 'PopulationOfLargestCityWithin20Miles'], inplace=True)
y = list(map(float, [str(ch).replace(" ", "").replace(",", ".") for ch in list(df.SalaryUSD)]))
X = df.drop(columns= "SalaryUSD")
X = X[["JobTitle", "OtherJobDuties", "KindsOfTasksPerformed", "CareerPlansThisYear"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = pipe.fit(X_train, y_train)
y_pred = clf.predict(X_test)
mean_squared_error(y_test, y_pred)

get_dummy_jobs start
done
get_dummy_jobs start
done
get_job_dummy start
done
get_dummy_jobs start
done
get_dummy_jobs start
done
get_job_dummy start
done


2470046594.109731

In [ ]:
Counter(X.CareerPlansThisYear)

Counter({'Change both employers and roles': 294,
         'Not Asked': 2898,
         'Prefer not to say': 347,
         'Stay with the same employer,  but change roles': 485,
         'Stay with the same employer, same role': 2398,
         'Stay with the same role, but change employers': 471})

In [ ]:
y_set = clf.predict(X_train)
Val = pd.DataFrame()
Val["y_train"] = y_train
Val["y_pred"] = y_set
Val

get_dummy_jobs start
done
get_dummy_jobs start
done
get_job_dummy start
done


,y_train,y_pred
0,55000.0,94528.518668
1,77000.0,94528.518668
2,116340.0,94014.059552
3,29200.0,94528.518668
4,182000.0,94528.518668
...,...,...
5509,700.0,94014.059552
5510,38000.0,94014.059552
5511,66000.0,94014.059552
5512,110000.0,94528.518668


In [ ]:
mean_squared_error(y_train, y_set)

In [ ]:
Val = pd.DataFrame()
Val["y_test"] = y_test
Val["y_pred"] = y_pred
Val

,y_test,y_pred
0,79000.0,91309.595112
1,35000.0,94014.059552
2,40500.0,91309.595112
3,65000.0,91309.595112
4,100000.0,94014.059552
...,...,...
1374,107000.0,94014.059552
1375,20000.0,94014.059552
1376,75000.0,94014.059552
1377,107000.0,94014.059552
